In [114]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

rs = 0

In [54]:
df = pd.read_csv("../data/npf_train.csv")
df.head()

,id,date,class4,partlybad,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,Glob.std,H2O168.mean,H2O168.std,H2O336.mean,H2O336.std,H2O42.mean,H2O42.std,H2O504.mean,H2O504.std,H2O672.mean,H2O672.std,H2O84.mean,H2O84.std,NET.mean,NET.std,NO168.mean,NO168.std,NO336.mean,NO336.std,NO42.mean,NO42.std,NO504.mean,NO504.std,NO672.mean,NO672.std,NO84.mean,NO84.std,NOx168.mean,NOx168.std,NOx336.mean,NOx336.std,NOx42.mean,NOx42.std,NOx504.mean,NOx504.std,NOx672.mean,NOx672.std,NOx84.mean,NOx84.std,O3168.mean,O3168.std,O342.mean,O342.std,O3504.mean,O3504.std,O3672.mean,O3672.std,O384.mean,O384.std,Pamb0.mean,Pamb0.std,PAR.mean,PAR.std,PTG.mean,PTG.std,RGlob.mean,RGlob.std,RHIRGA168.mean,RHIRGA168.std,RHIRGA336.mean,RHIRGA336.std,RHIRGA42.mean,RHIRGA42.std,RHIRGA504.mean,RHIRGA504.std,RHIRGA672.mean,RHIRGA672.std,RHIRGA84.mean,RHIRGA84.std,RPAR.mean,RPAR.std,SO2168.mean,SO2168.std,SWS.mean,SWS.std,T168.mean,T168.std,T42.mean,T42.std,T504.mean,T504.std,T672.mean,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std
0,1,2000-02-23,nonevent,False,380.528120,0.802001,380.371466,0.889550,381.816207,1.292593,380.296466,0.968884,236.605353,145.160571,2.663504,0.319427,2.695603,0.304894,2.548879,0.382813,2.694138,0.306606,2.771071,0.366386,2.613362,0.348996,81.699876,109.164607,0.319316,0.179649,0.336724,0.183974,0.235517,0.157543,0.332500,0.183478,0.287241,0.172559,0.286638,0.160141,2.657949,0.672278,2.650259,0.667324,2.621983,0.646127,2.642241,0.667192,2.609828,0.684998,2.645259,0.652994,32.624872,0.697921,31.269483,1.853033,32.866724,0.892441,32.675345,0.928274,32.235086,0.938220,1006.514940,0.217193,339.171515,211.125658,0.000964,0.007481,67.500843,30.529852,95.104103,1.612618,96.532586,2.382040,92.189052,1.781872,96.752672,2.326736,101.351071,4.571036,93.293534,1.981996,84.476919,49.764321,0.559316,0.374688,936.605263,0.916500,-10.272530,1.575415,-10.489828,2.085275,-10.346540,1.347401,-10.730843,1.381815,-10.282754,1.870056,8.356761,4.534937,0.178084,0.123402,0.002546,0.000686
1,2,2000-03-25,Ib,False,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,138.921953,3.253684,0.299728,3.232222,0.308108,3.299150,0.290206,3.228039,0.310416,3.227712,0.307517,3.267582,0.298119,142.534162,115.885107,0.023618,0.040306,0.028105,0.042142,0.025294,0.045379,0.027908,0.046759,0.030458,0.042894,0.025425,0.094298,0.843355,0.159896,0.829739,0.157876,0.914641,0.574305,0.824510,0.162646,0.833987,0.174832,0.864379,0.229341,47.953553,3.039311,47.584248,3.272690,48.135098,2.780791,48.196993,2.678606,47.717712,3.193208,992.994188,0.280526,487.596401,268.860550,-0.005340,0.008779,41.090445,21.317089,59.672237,14.625643,60.119150,14.369137,59.535033,15.592649,60.910915,14.122225,62.475294,14.217269,59.184771,15.260717,32.421126,19.516104,0.138289,0.115293,923.450980,2.062171,-1.330589,1.947330,-1.037435,2.231552,-1.738455,1.748079,-2.095641,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210
2,3,2000-04-06,Ib,False,372.363293,0.626329,372.245689,0.615803,372.847246,0.647279,372.193952,0.596289,269.981547,200.826676,4.459042,0.367894,4.422874,0.365105,4.509760,0.360563,4.409401,0.359296,4.395988,0.357929,4.462515,0.362899,156.409709,173.191387,0.030898,0.047893,0.030000,0.046062,0.028802,0.047107,0.024910,0.045468,0.029940,0.052406,0.028802,0.050870,0.747545,0.207766,0.736407,0.174350,0.775150,0.254238,0.705210,0.156536,0.709701,0.152953,0.771078,0.245791,45.974192,1.892847,45.661317,1.922014,46.141737,1.895122,46.116168,1.889977,45.854731,1.943906,986.740455,2.777462,515.622611,392.245819,-0.002910,0.006699,41.006814,29.483793,65.047844,13.978114,65.240539,14.181466,64.822395,13.827617,65.872216,14.174827,67.481018,14.652811,64.301856,13.820666,32.940216,25.385709,0.107066,0.122741,923.410714,2.647653,1.672108,1.942941,1.893257,1.960102,1.354012,1.910314,0.991521,1.914186,1.846503,1.954748,14.286261,9.572444,0.418313,0.344386

In [55]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [56]:
df = df.drop(columns=["partlybad", "id"])

In [57]:
df['month'] = pd.DatetimeIndex(df['date']).month
df = df.drop(columns=["date"])
df.head()

,class4,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,Glob.std,H2O168.mean,H2O168.std,H2O336.mean,H2O336.std,H2O42.mean,H2O42.std,H2O504.mean,H2O504.std,H2O672.mean,H2O672.std,H2O84.mean,H2O84.std,NET.mean,NET.std,NO168.mean,NO168.std,NO336.mean,NO336.std,NO42.mean,NO42.std,NO504.mean,NO504.std,NO672.mean,NO672.std,NO84.mean,NO84.std,NOx168.mean,NOx168.std,NOx336.mean,NOx336.std,NOx42.mean,NOx42.std,NOx504.mean,NOx504.std,NOx672.mean,NOx672.std,NOx84.mean,NOx84.std,O3168.mean,O3168.std,O342.mean,O342.std,O3504.mean,O3504.std,O3672.mean,O3672.std,O384.mean,O384.std,Pamb0.mean,Pamb0.std,PAR.mean,PAR.std,PTG.mean,PTG.std,RGlob.mean,RGlob.std,RHIRGA168.mean,RHIRGA168.std,RHIRGA336.mean,RHIRGA336.std,RHIRGA42.mean,RHIRGA42.std,RHIRGA504.mean,RHIRGA504.std,RHIRGA672.mean,RHIRGA672.std,RHIRGA84.mean,RHIRGA84.std,RPAR.mean,RPAR.std,SO2168.mean,SO2168.std,SWS.mean,SWS.std,T168.mean,T168.std,T42.mean,T42.std,T504.mean,T504.std,T672.mean,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,month
0,nonevent,380.528120,0.802001,380.371466,0.889550,381.816207,1.292593,380.296466,0.968884,236.605353,145.160571,2.663504,0.319427,2.695603,0.304894,2.548879,0.382813,2.694138,0.306606,2.771071,0.366386,2.613362,0.348996,81.699876,109.164607,0.319316,0.179649,0.336724,0.183974,0.235517,0.157543,0.332500,0.183478,0.287241,0.172559,0.286638,0.160141,2.657949,0.672278,2.650259,0.667324,2.621983,0.646127,2.642241,0.667192,2.609828,0.684998,2.645259,0.652994,32.624872,0.697921,31.269483,1.853033,32.866724,0.892441,32.675345,0.928274,32.235086,0.938220,1006.514940,0.217193,339.171515,211.125658,0.000964,0.007481,67.500843,30.529852,95.104103,1.612618,96.532586,2.382040,92.189052,1.781872,96.752672,2.326736,101.351071,4.571036,93.293534,1.981996,84.476919,49.764321,0.559316,0.374688,936.605263,0.916500,-10.272530,1.575415,-10.489828,2.085275,-10.346540,1.347401,-10.730843,1.381815,-10.282754,1.870056,8.356761,4.534937,0.178084,0.123402,0.002546,0.000686,2
1,Ib,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,138.921953,3.253684,0.299728,3.232222,0.308108,3.299150,0.290206,3.228039,0.310416,3.227712,0.307517,3.267582,0.298119,142.534162,115.885107,0.023618,0.040306,0.028105,0.042142,0.025294,0.045379,0.027908,0.046759,0.030458,0.042894,0.025425,0.094298,0.843355,0.159896,0.829739,0.157876,0.914641,0.574305,0.824510,0.162646,0.833987,0.174832,0.864379,0.229341,47.953553,3.039311,47.584248,3.272690,48.135098,2.780791,48.196993,2.678606,47.717712,3.193208,992.994188,0.280526,487.596401,268.860550,-0.005340,0.008779,41.090445,21.317089,59.672237,14.625643,60.119150,14.369137,59.535033,15.592649,60.910915,14.122225,62.475294,14.217269,59.184771,15.260717,32.421126,19.516104,0.138289,0.115293,923.450980,2.062171,-1.330589,1.947330,-1.037435,2.231552,-1.738455,1.748079,-2.095641,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,3
2,Ib,372.363293,0.626329,372.245689,0.615803,372.847246,0.647279,372.193952,0.596289,269.981547,200.826676,4.459042,0.367894,4.422874,0.365105,4.509760,0.360563,4.409401,0.359296,4.395988,0.357929,4.462515,0.362899,156.409709,173.191387,0.030898,0.047893,0.030000,0.046062,0.028802,0.047107,0.024910,0.045468,0.029940,0.052406,0.028802,0.050870,0.747545,0.207766,0.736407,0.174350,0.775150,0.254238,0.705210,0.156536,0.709701,0.152953,0.771078,0.245791,45.974192,1.892847,45.661317,1.922014,46.141737,1.895122,46.116168,1.889977,45.854731,1.943906,986.740455,2.777462,515.622611,392.245819,-0.002910,0.006699,41.006814,29.483793,65.047844,13.978114,65.240539,14.181466,64.822395,13.827617,65.872216,14.174827,67.481018,14.652811,64.301856,13.820666,32.940216,25.385709,0.107066,0.122741,923.410714,2.647653,1.672108,1.942941,1.893257,1.960102,1.354012,1.910314,0.991521,1.914186,1.846503,1.954748,14.286261,9.572444,0.418313,0.344386,0.000541,0.000072,4
3,nonevent,381.437442,7.281159,381.380405,7.

In [96]:
df["class2"] = df["class4"].apply(lambda x: 0 if x == "nonevent" else 1)

In [97]:
def accuracy(y_pred, y_test):
    correct = 0
    total = len(y_pred)

    for i in range(0, total):
        if y_pred[i] == y_test[i]:
            correct += 1

    return(correct / total) 

In [98]:
## Multi-class random forest

y = np.array(df["class4"])
X = np.array(df.drop(columns=["class4", "class2"]))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = rs)

rf = RandomForestClassifier(n_estimators = 1000, random_state = rs)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy(y_pred, y_test)

0.6759259259259259

In [122]:
## Single-class random forest

y = np.array(df["class2"])
X = np.array(df.drop(columns=["class4", "class2"]))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = rs)

rf = RandomForestClassifier(n_estimators = 1000, random_state = rs)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy(y_pred, y_test)

0.8333333333333334

In [121]:
y = np.array(df["class2"])
X = np.array(df.drop(columns=["class4", "class2"]))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = rs)

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
lr.score(X_test, y_test)


0.8518518518518519

In [112]:
y_pred

array([ 0.85418915,  1.13669101,  0.85249482, -0.26200624,  0.5307249 ,
        0.35823146,  0.37887595,  0.827728  , -0.415769  , -0.06067545,
        0.03669904,  0.87217381,  0.35214917,  1.25716402,  0.55593395,
        0.02656618,  0.20535169,  0.55746909,  0.15202609,  0.55449268,
        0.51713835,  0.97390541,  0.44056158,  1.09734081,  0.95957005,
       -0.00815656,  0.49508169,  0.61349287,  0.21410573,  1.64139829,
        0.08010537,  0.76036704,  0.6385631 ,  0.59979582,  0.31500588,
        0.40016873,  0.02371768,  0.01043672, -0.21708388, -0.11462728,
        0.78778114, -0.0701892 ,  0.79972937,  0.38228623,  0.38290132,
        0.00992864,  0.3728839 ,  0.66657837,  0.5782597 ,  0.76600349,
        0.94716944,  0.19492   ,  0.50766531,  0.83581742,  0.45379475,
        0.1613437 ,  0.71849916,  0.55171468,  1.01905199, -0.37406653,
       -0.19260464,  0.93390003,  0.65247008,  0.39755332,  0.76770394,
        0.14467889,  0.56056598,  0.18354865,  0.08771258,  0.68